In [2]:
import boto3
import pandas as pd; pd.set_option('display.max_columns', 100)
import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.metrics import recall_score, classification_report
from sklearn.impute import SimpleImputer

s3 = boto3.resource('s3')
bucket_name = 'grant-gonnerman-data-445'
bucket = s3.Bucket(bucket_name)

file_key = 'framingham.csv'

bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

# reading data file
heart = pd.read_csv(file_content_stream)
heart.head()

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


In [3]:
#defining input at target 
x = heart[['age', 'currentSmoker', 'totChol', 'BMI', 'heartRate']]
y = heart['TenYearCHD']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, stratify = y)

In [4]:
x_train.describe()

,age,currentSmoker,totChol,BMI,heartRate
count,3390.000000,3390.000000,3355.000000,3374.000000,3389.000000
mean,49.608260,0.492035,236.440238,25.842516,75.775745
std,8.568743,0.500010,44.453968,4.069442,12.014916
min,32.000000,0.000000,107.000000,15.540000,44.000000
25%,42.000000,0.000000,205.000000,23.080000,68.000000
50%,49.000000,0.000000,233.000000,25.430000,75.000000
75%,56.000000,1.000000,263.000000,28.060000,82.000000
max,70.000000,1.000000,696.000000,51.280000,143.000000


In [5]:
x_test.describe()

,age,currentSmoker,totChol,BMI,heartRate
count,848.000000,848.000000,833.000000,845.000000,848.000000
mean,49.491745,0.502358,237.854742,25.640260,76.291274
std,8.590235,0.500290,45.144578,4.120892,12.071464
min,33.000000,0.000000,119.000000,16.610000,48.000000
25%,42.000000,0.000000,207.000000,23.050000,68.000000
50%,49.000000,1.000000,235.000000,25.200000,75.000000
75%,56.000000,1.000000,264.000000,27.860000,85.000000
max,69.000000,1.000000,464.000000,56.800000,125.000000


In [7]:
# defining imputer
imputer = SimpleImputer(missing_values = np.nan, strategy = 'median')
imputer.fit(x_train)

# imputing missing values
x_train_imp = pd.DataFrame(imputer.transform(x_train), columns = x_train.columns)
x_test_imp = pd.DataFrame(imputer.transform(x_test), columns = x_test.columns)

In [8]:
x_train_imp.describe()

,age,currentSmoker,totChol,BMI,heartRate
count,3390.000000,3390.000000,3390.00000,3390.000000,3390.000000
mean,49.608260,0.492035,236.40472,25.840569,75.775516
std,8.568743,0.500010,44.22519,4.059923,12.013150
min,32.000000,0.000000,107.00000,15.540000,44.000000
25%,42.000000,0.000000,206.00000,23.080000,68.000000
50%,49.000000,0.000000,233.00000,25.430000,75.000000
75%,56.000000,1.000000,262.00000,28.060000,82.000000
max,70.000000,1.000000,696.00000,51.280000,143.000000


In [9]:
x_test_imp.describe()

,age,currentSmoker,totChol,BMI,heartRate
count,848.000000,848.000000,848.000000,848.000000,848.000000
mean,49.491745,0.502358,237.768868,25.639517,76.291274
std,8.590235,0.500290,44.747629,4.113606,12.071464
min,33.000000,0.000000,119.000000,16.610000,48.000000
25%,42.000000,0.000000,208.000000,23.050000,68.000000
50%,49.000000,1.000000,234.000000,25.220000,75.000000
75%,56.000000,1.000000,264.000000,27.830000,85.000000
max,69.000000,1.000000,464.000000,56.800000,125.000000


In [10]:
scaler = MinMaxScaler()

x_train_imp = pd.DataFrame(scaler.fit_transform(x_train_imp), columns = x_train_imp.columns)
x_test_imp = pd.DataFrame(scaler.fit_transform(x_test_imp), columns = x_test_imp.columns)

In [12]:
# logistic model
logit_md = LogisticRegression().fit(x_train_imp, y_train)

# predicting on test
logit_pred = logit_md.predict_proba(x_test_imp)[:,1]

# likilyhoods to labels
logit_label = np.where(logit_pred < 0.1, 0, 1)

print(classification_report(y_test, logit_label))

              precision    recall  f1-score   support

           0       0.94      0.36      0.52       719
           1       0.20      0.88      0.32       129

    accuracy                           0.44       848
   macro avg       0.57      0.62      0.42       848
weighted avg       0.83      0.44      0.49       848



In [13]:
# Random Forest model
rf_md = RandomForestClassifier(n_estimators = 500, max_depth = 5).fit(x_train_imp, y_train)

# predicting on test
rf_pred = rf_md.predict_proba(x_test_imp)[:,1]

# likilyhoods to labels
rf_label = np.where(rf_pred < 0.1, 0, 1)

print(classification_report(y_test, rf_label))

              precision    recall  f1-score   support

           0       0.95      0.23      0.37       719
           1       0.18      0.93      0.30       129

    accuracy                           0.34       848
   macro avg       0.56      0.58      0.34       848
weighted avg       0.83      0.34      0.36       848



In [ ]:
# from my results i would use the logistic regression model because is had better performace metics overall 